# ✨ Welcome to the OSCAR Usage Tutorial from a Jupyter Notebook 

In this documentation, we will guide you step-by-step to set up our environment and provide simple example use cases to help you understand the basics of working with OSCAR clusters from a Jupyter Notebook.  

This tutorial will focus on:
- How to set up the environment of the Jupyter Notebook.
- How to define an OSCAR service.
- How to create the service in the OSCAR cluster.
- How to call synchronous services.
- How to call asynchronous services.
- How to upload and download files from the service's bucket.
- How to delete services.

Let's get started! 🚀  


## 🔧 First, Set Up the Environment  

### Method 1 (Divace authentication)

To begin, open a terminal in Jupyter Notebook:  
📌 **File → New → Terminal**  

Then we need to execute `./setupscript.sh`. This script will configure the OpenID client and initialize an authentication process in which you will **obtain a device authentication code** that you need to insert on the EGI authentication portal. 

After obtaining the code:  
1️⃣ Go to [EGI Divace Authentication Portal](https://aai.egi.eu/device).  
2️⃣ Enter the provided code.  
3️⃣ Return to the terminal and enter your password twice. 

**_NOTE $^1$:_** The short name for the authenticated user by default is "oscar-egi".  
**_NOTE $^2$:_** If you can't find `./setupscript.sh`, execute `cd /`. 

### Mrthod 2 (Token)

Open the [EGI token page](https://aai.egi.eu/token/) to obtain an access token to use in this tutorial.

**_NOTE $^1$:_** The access token expires in 1 hour, so you will need to repeat the process.

## ⏭️ Next Steps  

In the following steps of this tutorial, we will provide guidance for two basic use cases:

✔️ **Synchronous Service**   
   - Receives an input   
   - Returns an output   

✔️ **Asynchronous Service** 
   - Triggers when a file is uploaded on the input storage (MinIO) 📂  
   - Processes it and stores the output to the same or another storage 📬 

**_NOTE:_** You can find more service examples in the [examples](https://github.com/grycap/oscar/tree/master/examples) section on GitHub. 